In [20]:
import numpy as np 
import pandas as pd 
import random
import matplotlib.pyplot as plt 
import os
pd.set_option('display.max_rows', 200)
%matplotlib inline

# Import Data 

In [27]:
# define folder where data resides 
DATAFOLDER = "/Users/Gary/Documents/data-science-coursework/nyu-ml/project/data"

### Appeals

In [28]:
# main table 
tblAppeal = pd.read_csv(os.path.join(DATAFOLDER, 'raw/tblAppeal.csv'), low_memory=False) 
print(tblAppeal.info())
tblAppeal.sample(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 870388 entries, 0 to 870387
Data columns (total 17 columns):
idnAppeal             870388 non-null int64
idncase               868758 non-null float64
idnProceeding         776380 non-null float64
strAppealCategory     870388 non-null object
strAppealType         870388 non-null object
datAppealFiled        870226 non-null object
strFiledBy            870378 non-null object
datAttorneyE27        521008 non-null object
datBIADecision        847196 non-null object
strBIADecision        847180 non-null object
strBIADecisionType    822882 non-null object
strCaseType           824855 non-null object
strLang               773811 non-null object
strNat                777004 non-null object
strProceedingIHP      367364 non-null object
strCustody            666356 non-null object
strProbono            1880 non-null object
dtypes: float64(2), int64(1), object(14)
memory usage: 112.9+ MB
None


,idnAppeal,idncase,idnProceeding,strAppealCategory,strAppealType,datAppealFiled,strFiledBy,datAttorneyE27,datBIADecision,strBIADecision,strBIADecisionType,strCaseType,strLang,strNat,strProceedingIHP,strCustody,strProbono
460571,4628201,2688973.0,934619.0,IJ,Case Appeal,2004-04-12 00:00:00,A,NaN,2005-07-12 00:00:00,SUD,P,DEP,ENG,DR,,R,NaN
652930,4826139,4436198.0,3011402.0,IJ,MTR BIA,2008-05-05 00:00:00,A,2008-05-05 00:00:00,2008-12-15 00:00:00,DEN,R,RMV,FOO,CH,,N,NaN
418355,4584732,3625540.0,1968662.0,IJ,Case Appeal,2003-06-26 00:00:00,A,2003-11-03 00:00:00,2004-08-17 00:00:00,REM,P,RMV,ENG,IN,,N,NaN
611763,4784459,3682235.0,3881059.0,IJ,Case Appeal,2007-05-09 00:00:00,A,2007-05-09 00:00:00,2009-06-05 00:00:00,DIS,R,RMV,RUS,AM,NaN,N,NaN
179195,183119,2507942.0,696763.0,IJ,Case Appeal,1997-01-21 00:00:00,A,NaN,1998-01-29 00:00:00,DIS,R,DEP,SP,ES,,D,NaN


In [29]:
# drop appeals with no case number, proceeding number, or decision 
tblAppeal.dropna(subset=['idncase', 'idnProceeding', 'strBIADecision'], inplace=True) 
tblAppeal = tblAppeal[tblAppeal['idnProceeding'] != 0] # drop zeros 

# convert indexes to integers 
tblAppeal['idncase'] = tblAppeal['idncase'].astype(int) 
tblAppeal['idnProceeding'] = tblAppeal['idnProceeding'].astype(int) 
tblAppeal.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 755222 entries, 0 to 869457
Data columns (total 17 columns):
idnAppeal             755222 non-null int64
idncase               755222 non-null int64
idnProceeding         755222 non-null int64
strAppealCategory     755222 non-null object
strAppealType         755222 non-null object
datAppealFiled        755060 non-null object
strFiledBy            755212 non-null object
datAttorneyE27        478717 non-null object
datBIADecision        755222 non-null object
strBIADecision        755222 non-null object
strBIADecisionType    731103 non-null object
strCaseType           755180 non-null object
strLang               752329 non-null object
strNat                754061 non-null object
strProceedingIHP      365943 non-null object
strCustody            643730 non-null object
strProbono            1842 non-null object
dtypes: int64(3), object(14)
memory usage: 103.7+ MB


In [30]:
# format column types 
tblAppeal['datAppealFiled_dt'] = pd.to_datetime(tblAppeal['datAppealFiled'], errors='coerce')
tblAppeal['datBIADecision_dt'] = pd.to_datetime(tblAppeal['datBIADecision'], errors='coerce')
tblAppeal['datAppealFiled_year'] = tblAppeal['datAppealFiled_dt'].dt.year

In [34]:
# code lookup tables 
bia_appeal_category = pd.read_excel(os.path.join(DATAFOLDER, 'raw/BIA Appeal Data File code translations.xlsx'), 
                                    sheet_name='BIA Appeal Category', skip_footer=7)
bia_appeal_type = pd.read_excel(os.path.join(DATAFOLDER, 'raw/BIA Appeal Data File code translations.xlsx'),
                                sheet_name='BIA Appeal Type', skip_footer=3)
bia_decision_type = pd.read_excel(os.path.join(DATAFOLDER, 'raw/BIA Appeal Data File code translations.xlsx'),
                                  sheet_name='BIA decision type', skip_footer=2)
bia_decision_code = pd.read_excel(os.path.join(DATAFOLDER, 'raw/BIA Appeal Data File code translations.xlsx'),
                                  sheet_name='BIA decision code', skip_footer=2)

In [35]:
# join them 
tblAppeal_df = tblAppeal.merge(bia_appeal_category, how='left', left_on='strAppealCategory', right_on='Code')\
                        .rename(columns={'Description': 'strAppealCategoryDesc'}).drop('Code', axis=1)\
                        .merge(bia_appeal_type, how='left', left_on='strAppealType', right_on='Code')\
                        .rename(columns={'Description': 'strAppealTypeDesc'}).drop('Code', axis=1)\
                        .merge(bia_decision_code, how='left', left_on='strBIADecision', right_on='Code')\
                        .rename(columns={'Description': 'strBIADecisionDesc'}).drop('Code', axis=1)\
                        .merge(bia_decision_type, how='left', left_on='strBIADecisionType', right_on='Code')\
                        .rename(columns={'Description': 'strBIADecisionTypeDesc'}).drop('Code', axis=1)
print(tblAppeal_df.info()) 
tblAppeal_df.sample(10).T

<class 'pandas.core.frame.DataFrame'>
Int64Index: 755222 entries, 0 to 755221
Data columns (total 24 columns):
idnAppeal                 755222 non-null int64
idncase                   755222 non-null int64
idnProceeding             755222 non-null int64
strAppealCategory         755222 non-null object
strAppealType             755222 non-null object
datAppealFiled            755060 non-null object
strFiledBy                755212 non-null object
datAttorneyE27            478717 non-null object
datBIADecision            755222 non-null object
strBIADecision            755222 non-null object
strBIADecisionType        731103 non-null object
strCaseType               755180 non-null object
strLang                   752329 non-null object
strNat                    754061 non-null object
strProceedingIHP          365943 non-null object
strCustody                643730 non-null object
strProbono                1842 non-null object
datAppealFiled_dt         755060 non-null datetime64[ns]
datB

,650854,601280,322378,497620,549549,444135,154521,332747,753357,170601
idnAppeal,4890338,4833545,4522733,4715808,4775547,4654867,177308,4534081,5036032,4345622
idncase,5426484,5929822,3813196,5186135,2892547,4243822,2556524,3385071,7277227,2588563
idnProceeding,3824863,4457929,2194401,3528518,1200189,2770729,759517,1685376,6275513,799646
strAppealCategory,IJ,IJ,IJ,IJ,IJ,IJ,IJ,IJ,IJ,IJ
strAppealType,MTR BIA,Case Appeal,Case Appeal,Case Appeal,MTR BIA,Case Appeal,Case Appeal,MTR BIA,Case Appeal,Case Appeal
datAppealFiled,2009-12-28 00:00:00,2008-07-09 00:00:00,2002-04-05 00:00:00,2005-11-22 00:00:00,2007-02-27 00:00:00,2004-10-12 00:00:00,1996-11-22 00:00:00,2002-06-25 00:00:00,2013-11-29 00:00:00,1997-05-06 00:00:00
strFiledBy,A,A,A,A,A,A,A,A,A,A
datAttorneyE27,2010-01-08 00:00:00,NaN,2002-04-05 00:00:00,2005-11-22 00:00:00,2007-02-27 00:00:00,2004-10-12 00:00:00,1996-11-22 00:00:00,2002-06-25 00:00:00,2013-11-29 00:00:00,1997-08-21 00:00:00
datBIADecision,2010-05-25 00:00:00,2008-09-12 00:00:00,2003-08-20 00:00:00,2006-11-09 00:00:00,2007-06-28 00:00:00,2005-12-01 00:00:00,1997-09-26 00:00:00,2003-01-10 00:00:00,2014-03-13 00:00:00,1998-06-22 00:00:00
strBIADecision,REM,DIS,D30,DVD,REM,DIS,D30,OTH,DIS,REM


In [36]:
# some strBIADecision don't have corresponding code translations; to remove 
print(tblAppeal_df[pd.isnull(tblAppeal_df['strBIADecisionDesc'])]['strBIADecision'].value_counts()) 
tblAppeal_df.dropna(subset=['strBIADecisionDesc'], inplace=True)
tblAppeal_df.info()

DSO    5987
DED    2107
CPG    1704
ABC    1606
APD    1170
GRS     431
DNS     421
ADM     355
DMO     238
CPC      90
MB       88
SNC      82
AFD      54
RET      52
ADD      18
TPD      16
CGR      16
REV      13
WPD       5
SUP       4
          2
Name: strBIADecision, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 740763 entries, 0 to 755221
Data columns (total 24 columns):
idnAppeal                 740763 non-null int64
idncase                   740763 non-null int64
idnProceeding             740763 non-null int64
strAppealCategory         740763 non-null object
strAppealType             740763 non-null object
datAppealFiled            740609 non-null object
strFiledBy                740753 non-null object
datAttorneyE27            470111 non-null object
datBIADecision            740763 non-null object
strBIADecision            740763 non-null object
strBIADecisionType        717284 non-null object
strCaseType               740721 non-null object
strLang          

### Master Proceedings

In [39]:
master = pd.read_csv(os.path.join(DATAFOLDER, 'raw/master.csv')) 
print(master.info())
master.sample(5)

/Users/Gary/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6084437 entries, 0 to 6084436
Data columns (total 16 columns):
idncase             float64
nat                 object
case_type           object
c_asy_type          object
idnproceeding       object
base_city_code      object
hearing_loc_code    object
dec_type            object
dec_code            object
other_comp          object
osc_date            object
input_date          object
comp_date           object
attorney_flag       float64
ij_code             object
tracid              float64
dtypes: float64(3), object(13)
memory usage: 742.7+ MB
None


,idncase,nat,case_type,c_asy_type,idnproceeding,base_city_code,hearing_loc_code,dec_type,dec_code,other_comp,osc_date,input_date,comp_date,attorney_flag,ij_code,tracid
1597656,3279950.0,MX,RMV,NaN,59917,SND,SDE,O,X,NaN,22AUG1997,25AUG1997,26AUG1997,NaN,GNB,260.0
3785439,5330559.0,GT,RMV,NaN,3709980,HOD,HOD,NaN,NaN,T,30AUG2005,07SEP2005,06OCT2005,NaN,JLB,66.0
2419340,3931081.0,MX,RMV,NaN,2347564,HAR,HAR,NaN,NaN,C,01JUN1999,29SEP1999,13OCT1999,NaN,JDC,181.0
4703068,6104722.0,MX,RMV,NaN,4678084,ELO,EAZ,W,X,NaN,12NOV2008,12NOV2008,12NOV2008,NaN,RAP,327.0
4363044,5819605.0,MX,RMV,NaN,4319370,BUF,BUF,O,V,NaN,16AUG2007,04SEP2007,25OCT2007,NaN,MR,24.0


In [40]:
# check missing data 
master.isnull().sum()

idncase                  14
nat                   16775
case_type                15
c_asy_type          4449366
idnproceeding             0
base_city_code           24
hearing_loc_code       1516
dec_type            1519549
dec_code            1674748
other_comp          4744870
osc_date              33353
input_date            29744
comp_date            335076
attorney_flag       3044591
ij_code               39598
tracid               341914
dtype: int64

In [45]:
# look up tables 
master_case_type = pd.read_csv(os.path.join(DATAFOLDER, 'raw/master_case_type.csv'))
master_decision_type = pd.read_csv(os.path.join(DATAFOLDER, 'raw/master_decision_type.csv'))
master_decision_on_proceeding = pd.read_csv(os.path.join(DATAFOLDER, 'raw/master_decision_on_proceeding.csv'), 
                                            skipfooter=2, engine='python')
master_decision_on_proceeding.drop(['Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6'], axis=1, inplace=True)

In [46]:
# add column to indicate whether decision was listed in dec_code or other_comp 
master['dec_col'] = np.where(pd.notnull(master['dec_code']), 'C', 'O') 
master['dec_judge'] = np.where(pd.notnull(master['dec_code']), master['dec_code'], master['other_comp']) 

In [47]:
# join to look up tables
master_df = master.merge(master_decision_type, how='left', left_on='dec_type', right_on='strCode')\
                  .rename(columns={'strDescription': 'dec_type_str'}).drop('strCode', axis=1)\
                  .merge(master_case_type, how='left', left_on='case_type', right_on='strCode')\
                  .rename(columns={'strDescription': 'case_type_str'}).drop('strCode', axis=1)\
                  .merge(master_decision_on_proceeding, how='left', left_on=['case_type', 'dec_judge', 'dec_col'], 
                         right_on=['strCaseType', 'strDecCode', 'strDecType'])\
                  .rename(columns={'strDecDescription': 'dec_code_str'}).drop(['strCaseType', 'strDecCode', 'strDecType'], axis=1)
master_df.sample(5)

,idncase,nat,case_type,c_asy_type,idnproceeding,base_city_code,hearing_loc_code,dec_type,dec_code,other_comp,...,input_date,comp_date,attorney_flag,ij_code,tracid,dec_col,dec_judge,dec_type_str,case_type_str,dec_code_str
4484343,5920707.0,GV,RMV,E,4778548,BOS,BOS,NaN,NaN,C,...,09MAR2009,03APR2009,1.0,MJD,16.0,O,C,NaN,Removal,Change of Venue
2074612,3666892.0,BE,RMV,I,2017295,NYC,NYC,O,X,NaN,...,13FEB1998,18DEC1998,1.0,SMB,138.0,C,X,oral decision,Removal,Remove
4743945,6138765.0,GT,RMV,NaN,4720954,KRO,BTC,W,X,NaN,...,17DEC2008,17DEC2008,NaN,RJF,115.0,C,X,written decision,Removal,Remove
3636492,5196818.0,CH,RMV,E,3597767,NYC,NYC,O,R,NaN,...,24MAY2005,13DEC2006,1.0,DBS,154.0,C,R,oral decision,Removal,Relief Granted
5640969,6869534.0,MX,RMV,NaN,5671196,TUC,TIN,O,V,NaN,...,18OCT2011,18OCT2011,NaN,TMO,49.0,C,V,oral decision,Removal,Voluntary Departure


### Master Proceedings (processed data from Daniel Chen)

In [50]:
master_dchen = pd.read_stata(os.path.join(DATAFOLDER,'decision_sched_merge_adjdate.dta')) # fails on Py3 for large files? 
master_dchen.info()

OSError: [Errno 22] Invalid argument

In [15]:
# check for nulls 
master_dchen.isnull().sum()

idncase                                 0
nat                                     0
case_type                               0
c_asy_type                              0
idnproceeding                           0
base_city_code                          0
hearing_loc_code                        0
dec_type                                0
dec_code                                0
other_comp                              0
attorney_flag                     2907462
ij_code                                 0
tracid                             248615
comp_year                               0
comp_month                              0
comp_day                                0
comp_date                               0
osc_year                            32420
osc_month                           32420
osc_day                             32420
osc_date                            32420
input_year                          27549
input_month                         27549
input_day                         

In [16]:
master_dchen.sample(3).T

,4494264,2715672,3057914
idncase,6687724,4402533,4907282
nat,MX,JM,CS
case_type,RMV,RMV,RMV
c_asy_type,,,
idnproceeding,5422859,2969197,3330761
base_city_code,LVG,ORL,ELP
hearing_loc_code,LVG,BDC,ELP
dec_type,O,,O
dec_code,X,,X
other_comp,,T,


In [17]:
# add column to indicate whether decision was listed in dec_code or other_comp 
master_dchen['dec_col'] = np.where(pd.notnull(master_dchen['dec_code']), 'C', 'O') 
master_dchen['dec_judge'] = np.where(pd.notnull(master_dchen['dec_code']), master_dchen['dec_code'], master_dchen['other_comp']) 

In [18]:
# join to look up tables
master_dchen_df = master_dchen.merge(master_decision_type, how='left', left_on='dec_type', right_on='strCode')\
                              .rename(columns={'strDescription': 'dec_type_str'}).drop('strCode', axis=1)\
                              .merge(master_case_type, how='left', left_on='case_type', right_on='strCode')\
                              .rename(columns={'strDescription': 'case_type_str'}).drop('strCode', axis=1)\
                              .merge(master_decision_on_proceeding, how='left', 
                                     left_on=['case_type', 'dec_judge', 'dec_col'], 
                                     right_on=['strCaseType', 'strDecCode', 'strDecType'])\
                              .rename(columns={'strDecDescription': 'dec_code_str'}).drop(['strCaseType', 'strDecCode', 'strDecType'], axis=1)
master_dchen_df.sample(3).T

,5552120,5249888,3846132
idncase,6806458,5884085,5829605
nat,SL,GT,GT
case_type,RMV,RMV,RMV
c_asy_type,,,I
idnproceeding,5583990,4400190,4331762
base_city_code,BAL,ATL,MIA
hearing_loc_code,BAL,ATD,MIA
dec_type,W,,O
dec_code,,,X
other_comp,A,T,


### Master Proceedings (processed by Sagent/Dunn)

In [54]:
master_dunn = pd.read_csv(os.path.join(DATAFOLDER, 
                                       'AsylumAdj/data_for_model/_decision_scheduling_merge_final_converted.csv'), 
                          encoding='latin-1', low_memory=False) # gets UnicodeDecodeError otherwise 
master_dunn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602500 entries, 0 to 602499
Columns: 182 entries, Unnamed: 0 to last_hearing_on_comp_date
dtypes: bool(1), float64(72), int64(39), object(70)
memory usage: 832.6+ MB


In [20]:
# dunn has far fewer proceedings than the original 
print "Original master has {} data points, Dunn's has {}".format(len(master_dunn), len(master))

Original master has 602500 data points, Dunn's has 6084437


In [52]:
print master_dchen.columns.tolist()

['idncase', 'nat', 'case_type', 'c_asy_type', 'idnproceeding', 'base_city_code', 'hearing_loc_code', 'dec_type', 'dec_code', 'other_comp', 'attorney_flag', 'ij_code', 'tracid', 'comp_year', 'comp_month', 'comp_day', 'comp_date', 'osc_year', 'osc_month', 'osc_day', 'osc_date', 'input_year', 'input_month', 'input_day', 'input_date', 'lawyer', 'defensive', 'affirmative', 'case_type_string', '_mcase', 'dec_type_string', '_mdectype', 'outcome_recorded_in_field', 'dec_string', '_mdecproceeddec', '_mdecproceedoth', 'nat_string', '_mnat', 'base_city_street', 'base_city_string', 'base_city_state', 'base_city_zip5', 'base_city_zip4', 'base_city_phone', '_mbasecity', 'hearing_loc_string1', 'hearing_loc_string2', 'hearing_loc_street', 'hearing_loc_city', 'hearing_loc_state', 'hearing_loc_zip5', 'hearing_loc_phone', '_mhearingloc', 'judge_name_caps', '_mlookupijcode', 'cityid', 'judgeid', 'natid', 'venue_change', 'deport', 'relief_granted', 'remove', 'terminated', 'voluntary_departure', 'oral', 'wr

In [53]:
master_dchen['asylum_only_proceeding'].value_counts()

0.0    5645688
1.0      18953
Name: asylum_only_proceeding, dtype: int64

In [56]:
master_dunn['case_type_string'].value_counts()

REMOVAL                 408738
DEPORTATION             142533
EXCLUSION                39062
ASYLUM ONLY CASE          9669
WITHHOLDING ONLY          2487
CREDIBLE FEAR REVIEW         1
Name: case_type_string, dtype: int64

In [59]:
master_dunn['asylum'].value_counts()

1    562348
0     40152
Name: asylum, dtype: int64

In [60]:
# determine what % of master proceedings that Dunn retained 
master_dunn_pairs = master_dunn[['idncase', 'idnproceeding']].copy()
master_dunn_pairs['in_dunn'] = 1 
master_dunn_delta = master_df[['idncase', 'idnproceeding', 'case_type_str']].merge(master_dunn_pairs, how='left', 
                                                                                   on=['idncase', 'idnproceeding']) 
master_dunn_delta['in_dunn'].fillna(0, inplace=True)
master_dunn_delta.groupby(['case_type_str'])['in_dunn'].agg(['size', np.mean])

,size,mean
case_type_str,,
Asylum Only Case,19881,0.447261
Claimed Status Review,1262,0.000000
Continued Detention Review,42,0.000000
Credible Fear Review,7453,0.000134
Deportation,1429597,0.099371
Exclusion,258052,0.151125
NACARA Adjustment,470,0.000000
Reasonable Fear Case,3611,0.000000
Recission,2087,0.000000


# Determine scope of appeal case types

In [65]:
appeals_in_scope = tblAppeal_df.merge(master_dunn_pairs, how='left', 
                                      left_on=['idncase', 'idnProceeding'], right_on=['idncase', 'idnproceeding']) 
appeals_in_scope = appeals_in_scope.merge(master_dchen[['idncase', 'idnproceeding', 'case_type_string']], 
                                          how='left', left_on=['idncase', 'idnProceeding'], 
                                          right_on=['idncase', 'idnproceeding']) 
appeals_in_scope['in_dunn'].fillna(0, inplace=True)
appeals_in_scope['case_type_string'].fillna('NOT_MATCHED_TO_MASTER', inplace=True)

In [68]:
# distribution of appeal vs. proceeding types - all
appeals_in_scope.groupby(['strAppealTypeDesc', 'case_type_string']).size().unstack().fillna(0) 

case_type_string,,ASYLUM ONLY CASE,CLAIMED STATUS REVIEW,CONTINUED DETENTION REVIEW,CREDIBLE FEAR REVIEW,DEPORTATION,EXCLUSION,NACARA ADJUSTMENT,NOT_MATCHED_TO_MASTER,REASONABLE FEAR CASE,RECISSION,REMOVAL,WITHHOLDING ONLY
strAppealTypeDesc,,,,,,,,,,,,,
Appeal of IJ MTR,3.0,168.0,1.0,0.0,0.0,16298.0,2058.0,4.0,358.0,0.0,6.0,21347.0,12.0
Bond Appeal,1.0,19.0,0.0,0.0,1.0,5511.0,14.0,0.0,1589.0,1.0,0.0,15259.0,6.0
Bond MTR Reconsider,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,11.0,0.0,0.0,84.0,0.0
Bond MTR Reinstate,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,25.0,0.0
Bond MTR Reopen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,46.0,0.0
Case Appeal,10.0,4251.0,9.0,0.0,101.0,140490.0,27977.0,56.0,9157.0,87.0,380.0,322863.0,974.0
Circuit Court Remand,0.0,197.0,0.0,0.0,0.0,1170.0,208.0,0.0,55.0,0.0,0.0,11134.0,27.0
Interlocutory Appeal,1.0,20.0,0.0,0.0,0.0,633.0,297.0,0.0,328.0,2.0,5.0,2101.0,9.0
MTR BIA,2.0,1347.0,0.0,5.0,4.0,40877.0,7473.0,8.0,1427.0,5.0,24.0,104105.0,93.0


In [71]:
# distribution of appeal vs. proceeding types - what dunn included 
appeals_in_scope[appeals_in_scope['in_dunn'] == 1].groupby(['strAppealTypeDesc', 'case_type_string']).size().unstack().fillna(0) 

case_type_string,,ASYLUM ONLY CASE,DEPORTATION,EXCLUSION,REMOVAL,WITHHOLDING ONLY
strAppealTypeDesc,,,,,,
Appeal of IJ MTR,0.0,93.0,2653.0,478.0,3026.0,9.0
Bond Appeal,0.0,10.0,569.0,7.0,2388.0,6.0
Bond MTR Reconsider,0.0,0.0,0.0,0.0,12.0,0.0
Bond MTR Reinstate,0.0,0.0,0.0,0.0,4.0,0.0
Bond MTR Reopen,0.0,0.0,0.0,0.0,10.0,0.0
Case Appeal,2.0,3878.0,66912.0,18341.0,173865.0,842.0
Circuit Court Remand,0.0,186.0,604.0,145.0,7058.0,26.0
Interlocutory Appeal,1.0,7.0,68.0,41.0,486.0,7.0
MTR BIA,0.0,1231.0,21833.0,5739.0,59309.0,87.0


#### Final Criteria: appeal types in ['Appeal of IJ MTR', 'Case Appeal', 'Circuit Court Remand', 'Interlocutory Appeal', 'MTR BIA'] and is found in Dunn's 

In [72]:
# filter relevant appeal types 
selected_appeal_types = ['Appeal of IJ MTR', 'Case Appeal', 'Circuit Court Remand', 'Interlocutory Appeal', 'MTR BIA'] 
appeals_in_scope = appeals_in_scope[appeals_in_scope['strAppealTypeDesc'].isin(selected_appeal_types) & 
                                    appeals_in_scope['in_dunn'] == 1].copy() 
appeals_in_scope.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366927 entries, 3 to 740756
Data columns (total 28 columns):
idnAppeal                 366927 non-null int64
idncase                   366927 non-null int64
idnProceeding             366927 non-null int64
strAppealCategory         366927 non-null object
strAppealType             366927 non-null object
datAppealFiled            366863 non-null object
strFiledBy                366918 non-null object
datAttorneyE27            255225 non-null object
datBIADecision            366927 non-null object
strBIADecision            366927 non-null object
strBIADecisionType        356952 non-null object
strCaseType               366926 non-null object
strLang                   366119 non-null object
strNat                    366752 non-null object
strProceedingIHP          183004 non-null object
strCustody                309504 non-null object
strProbono                750 non-null object
datAppealFiled_dt         366863 non-null datetime64[ns]
datBI

# Assign Labels 

In [73]:
# many different labels 
appeals_in_scope.groupby(['strBIADecisionDesc', 'strAppealTypeDesc']).size().unstack().fillna(0)

strAppealTypeDesc,Appeal of IJ MTR,Case Appeal,Circuit Court Remand,Interlocutory Appeal,MTR BIA
strBIADecisionDesc,,,,,
Administrative Return,0.0,11.0,0.0,0.0,10.0
Background Check Remand,17.0,4367.0,588.0,0.0,117.0
Continued,2.0,1191.0,23.0,0.0,23.0
Denied,4.0,24.0,29.0,0.0,55687.0
Dismiss Appeal/Affirm IJ's Decision,3304.0,96072.0,770.0,209.0,16.0
Dismiss as Untimely,102.0,1761.0,0.0,1.0,28.0
Dismissed (Grant V/D 30 days),3.0,30534.0,1.0,2.0,19.0
Dismissed (Voluntary Departure Granted),24.0,11259.0,198.0,0.0,0.0
Grant With No Remand,0.0,0.0,0.0,0.0,983.0


In [74]:
# assign positive and negative labels
granted_decisions = ['Background Check Remand', 'Grant With No Remand', 'Granted', 'Remand', 
                     'Sustain', 'Temporary Protected Status']
denied_decisions = ['Denied', "Dismiss Appeal/Affirm IJ's Decision", 'Dismissed (Grant V/D 30 days)', 
                    'Dismissed (Voluntary Departure Granted)', 'Rejection', 'SUMMARY AFFIRMANCE/VD', 
                    'Summary Affirmance', 'Summary Dismiss', 'Summary Dismissal (O) Other', 
                    'Summary Dismissal (a) inad reason on appeal', 'Summary Dismissal - Both (a) & (e)']
appeals_in_scope['granted'] = np.where(appeals_in_scope['strBIADecisionDesc'].isin(granted_decisions), 1, 
                                       np.where(appeals_in_scope['strBIADecisionDesc'].isin(denied_decisions), 0, None))

# drop neutral labels where appeals were dismissed due to administrative/procedural failures 
appeals_in_scope.dropna(subset=['granted'], inplace=True)
appeals_in_scope['granted'] = appeals_in_scope['granted'].astype(int)
appeals_in_scope.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 336976 entries, 3 to 740753
Data columns (total 29 columns):
idnAppeal                 336976 non-null int64
idncase                   336976 non-null int64
idnProceeding             336976 non-null int64
strAppealCategory         336976 non-null object
strAppealType             336976 non-null object
datAppealFiled            336916 non-null object
strFiledBy                336967 non-null object
datAttorneyE27            237308 non-null object
datBIADecision            336976 non-null object
strBIADecision            336976 non-null object
strBIADecisionType        329045 non-null object
strCaseType               336975 non-null object
strLang                   336254 non-null object
strNat                    336825 non-null object
strProceedingIHP          166763 non-null object
strCustody                285583 non-null object
strProbono                691 non-null object
datAppealFiled_dt         336916 non-null datetime64[ns]
datBI

# Dedup for unique (idncase, idnproceeding)

~24% of remaining proceedings have multiple appeals 

In [75]:
appeals_per_proceeding = appeals_in_scope.groupby(['idnProceeding'])['idnAppeal'].nunique()
appeals_per_proceeding.value_counts(normalize=True)

1     0.756449
2     0.184694
3     0.042579
4     0.011327
5     0.003151
6     0.001162
7     0.000351
8     0.000106
9     0.000051
15    0.000043
10    0.000024
13    0.000020
11    0.000016
12    0.000008
20    0.000008
21    0.000004
14    0.000004
23    0.000004
Name: idnAppeal, dtype: float64

Most multiple appeals seem to involve 'MTR' after initial 'Case Appeal'. Should we use the last instance of 'MTR' as the final decision?

In [76]:
# examples of proceedings with multiple appeals 
sample_index = random.sample(appeals_per_proceeding[appeals_per_proceeding > 1].index, 1)
tblAppeal_df[tblAppeal_df['idnProceeding'].isin(sample_index)].T 

,306207,363526
idnAppeal,4504808,4567207
idncase,4084134,4084134
idnProceeding,2555620,2555620
strAppealCategory,IJ,IJ
strAppealType,Case Appeal,MTR BIA
datAppealFiled,2001-10-03 00:00:00,2003-02-19 00:00:00
strFiledBy,A,A
datAttorneyE27,2001-10-03 00:00:00,2003-08-22 00:00:00
datBIADecision,2002-12-11 00:00:00,2003-09-23 00:00:00
strBIADecision,SAF,DEN


In [77]:
# check original proceedings for a given case 
master_dchen_df[master_dchen_df['idncase'] == 2241734].T

,4812094
idncase,2241734
nat,EG
case_type,DEP
c_asy_type,E
idnproceeding,361062
base_city_code,NEW
hearing_loc_code,NEW
dec_type,O
dec_code,V
other_comp,


~2% of cases have multiple proceedings 

In [79]:
proceedings_per_case = appeals_in_scope.groupby(['idncase'])['idnProceeding'].nunique()
proceedings_per_case.value_counts(normalize=True)

1    0.978357
2    0.020922
3    0.000713
4    0.000008
Name: idnProceeding, dtype: float64

In [80]:
# examples of cases with multiple proceedings 
sample_index = random.sample(proceedings_per_case[proceedings_per_case > 1].index, 1)
appeals_in_scope[appeals_in_scope['idncase'].isin(sample_index)].sort_values(by='datAppealFiled_dt').T 

,164312,278591
idnAppeal,4348577,4486150
idncase,2962678,2962678
idnProceeding,1291597,1291596
strAppealCategory,IJ,IJ
strAppealType,Case Appeal,Case Appeal
datAppealFiled,1997-05-28 00:00:00,2001-03-26 00:00:00
strFiledBy,A,A
datAttorneyE27,2000-04-24 00:00:00,2001-03-26 00:00:00
datBIADecision,2000-06-22 00:00:00,2003-11-17 00:00:00
strBIADecision,REM,DIS


In [81]:
# check original proceedings for a given case 
master_df[master_df['idncase'] == 5612569].T

,4112806,4112807,4112808,4112809,4112810
idncase,5.61257e+06,5.61257e+06,5.61257e+06,5.61257e+06,5.61257e+06
nat,ZA,ZA,ZA,ZA,ZA
case_type,RMV,RMV,RMV,RMV,RMV
c_asy_type,NaN,NaN,NaN,NaN,NaN
idnproceeding,4054290,4066075,4093297,4370491,5116381
base_city_code,FLO,PHO,ATL,ATL,ATL
hearing_loc_code,FLO,PHO,ATL,ATL,ATL
dec_type,NaN,NaN,O,NaN,O
dec_code,NaN,NaN,X,NaN,T
other_comp,T,C,NaN,O,NaN


** For each unique (case,proceeding) pairs with more than 1 appeal, pick the last appeal and dedup the rest ** 

In [82]:
appeals_deduped = appeals_in_scope.sort_values(by=['idncase', 'idnProceeding', 'datBIADecision_dt'], 
                                               ascending=[True, True, False])
appeals_deduped.drop_duplicates(subset=['idncase', 'idnProceeding'], keep='first', inplace=True)
appeals_deduped.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253906 entries, 433199 to 731825
Data columns (total 29 columns):
idnAppeal                 253906 non-null int64
idncase                   253906 non-null int64
idnProceeding             253906 non-null int64
strAppealCategory         253906 non-null object
strAppealType             253906 non-null object
datAppealFiled            253850 non-null object
strFiledBy                253901 non-null object
datAttorneyE27            178834 non-null object
datBIADecision            253906 non-null object
strBIADecision            253906 non-null object
strBIADecisionType        247517 non-null object
strCaseType               253906 non-null object
strLang                   253572 non-null object
strNat                    253789 non-null object
strProceedingIHP          123803 non-null object
strCustody                212262 non-null object
strProbono                578 non-null object
datAppealFiled_dt         253850 non-null datetime64[ns]


**31% of appeals were granted**

In [84]:
# check % granted in final dataset 
appeals_deduped['granted'].value_counts()

0    175095
1     78811
Name: granted, dtype: int64

# Merge Datasets

#### Add variables from master-chen

In [167]:
# variables in processed data by Prof. Chen
print(master_dchen.columns.tolist())

['idncase', 'nat', 'case_type', 'c_asy_type', 'idnproceeding', 'base_city_code', 'hearing_loc_code', 'dec_type', 'dec_code', 'other_comp', 'attorney_flag', 'ij_code', 'tracid', 'comp_year', 'comp_month', 'comp_day', 'comp_date', 'osc_year', 'osc_month', 'osc_day', 'osc_date', 'input_year', 'input_month', 'input_day', 'input_date', 'lawyer', 'defensive', 'affirmative', 'case_type_string', '_mcase', 'dec_type_string', '_mdectype', 'outcome_recorded_in_field', 'dec_string', '_mdecproceeddec', '_mdecproceedoth', 'nat_string', '_mnat', 'base_city_street', 'base_city_string', 'base_city_state', 'base_city_zip5', 'base_city_zip4', 'base_city_phone', '_mbasecity', 'hearing_loc_string1', 'hearing_loc_string2', 'hearing_loc_street', 'hearing_loc_city', 'hearing_loc_state', 'hearing_loc_zip5', 'hearing_loc_phone', '_mhearingloc', 'judge_name_caps', '_mlookupijcode', 'cityid', 'judgeid', 'natid', 'venue_change', 'deport', 'relief_granted', 'remove', 'terminated', 'voluntary_departure', 'oral', 'wr

In [112]:
master_dchen.groupby(['grantordeny_chicago']).size()

grantordeny_chicago
1.0    609491
dtype: int64

In [87]:
master_dchen.isnull().sum()

idncase                                 0
nat                                     0
case_type                               0
c_asy_type                              0
idnproceeding                           0
base_city_code                          0
hearing_loc_code                        0
dec_type                                0
dec_code                                0
other_comp                              0
attorney_flag                     2907462
ij_code                                 0
tracid                             248615
comp_year                               0
comp_month                              0
comp_day                                0
comp_date                               0
osc_year                            32420
osc_month                           32420
osc_day                             32420
osc_date                            32420
input_year                          27549
input_month                         27549
input_day                         

In [169]:
master_dchen.sample(4).T

,1825326,1887523,1580690,5094761
idncase,3669309,3721250,3449864,4484207
nat,CH,CH,MX,IN
case_type,RMV,RMV,DEP,RMV
c_asy_type,I,I,I,
idnproceeding,2020253,2083270,1761791,3068358
base_city_code,LOS,NYC,LOS,NOL
hearing_loc_code,LOS,NYX,LOS,NOO
dec_type,O,O,O,
dec_code,R,X,D,
other_comp,,,,T


In [113]:
dchen_selected_features = ['idncase', 'idnproceeding', 'nat_string', 'ij_code', 'attorney_flag',
                           'lawyer', 'defensive', 'affirmative', 
                           'oral', 'written', 'base_city_code', 'base_city_state', 
                           'hearing_loc_city', 'hearing_loc_code', 'hearing_loc_state', 'venue_change', 
                           'comp_year', 'comp_month', 'comp_day', 
                           'osc_year', 'osc_month', 'osc_day', 'input_year', 'input_month', 'input_day',
                           'adj_time_start', 'flag_earlystarttime', 
                           'flag_mismatch_base_city', 'flag_mismatch_hearing', 'flag_datemismatch',
                           'case_type_string', 'dec_type_string', 'dec_string', 
                           'deport', 'relief_granted', 'remove', 'terminated', 'voluntary_departure', 
                           'deport_form', 'voluntary_form', 'deportation_proceeding', 
                           'exclusion_proceeding', 'removal_proceeding', 'asylum_only_proceeding', 
                           'withholding_only_proceeding'] 

In [135]:
# merge with master proceedings 
appeals_with_master = appeals_deduped.merge(master_dchen[dchen_selected_features], how='left', 
                                            left_on=['idncase', 'idnProceeding'], right_on=['idncase', 'idnproceeding'])\
                                     .drop(['idnproceeding_y', 'case_type_string_y'], axis=1)\
                                     .rename(columns={'idnproceeding_x': 'idnproceeding', 
                                                      'case_type_string_x': 'case_type_string'})
appeals_with_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 253906 entries, 0 to 253905
Data columns (total 71 columns):
idnAppeal                      253906 non-null int64
idncase                        253906 non-null int64
idnProceeding                  253906 non-null int64
strAppealCategory              253906 non-null object
strAppealType                  253906 non-null object
datAppealFiled                 253850 non-null object
strFiledBy                     253901 non-null object
datAttorneyE27                 178834 non-null object
datBIADecision                 253906 non-null object
strBIADecision                 253906 non-null object
strBIADecisionType             247517 non-null object
strCaseType                    253906 non-null object
strLang                        253572 non-null object
strNat                         253789 non-null object
strProceedingIHP               123803 non-null object
strCustody                     212262 non-null object
strProbono                  

In [134]:
appeals_with_master.groupby(['case_type_string']).size()

case_type_string
                         2
ASYLUM ONLY CASE      3744
DEPORTATION          63442
EXCLUSION            17211
REMOVAL             168746
WITHHOLDING ONLY       761
dtype: int64

#### Add hearing sessions data

In [117]:
# TODO

#### Add judge bios data 

In [118]:
# TODO 

#### Output dataset for model training

In [139]:
# define features to output for training 
output_features = ['idnAppeal', 'nat_string', 'strCustody', 'strProbono', 'case_type_string',
                   'ij_code', 'lawyer', 'defensive', 'affirmative', 'oral', 'written', 
                   'comp_year', 'osc_year', 'input_year', 'datAppealFiled_year', 'granted']
appeals_final = appeals_with_master[output_features].copy()

# define mandatory features, without which data will be dropped 
mandatory_features = ['idnAppeal', 'granted', 'datAppealFiled_year', 'case_type_string'] 
appeals_final.dropna(subset=mandatory_features, inplace=True)

# generate output 
appeals_final.to_csv('data_for_model/data_for_model_2018-03-23.csv', index=False)